### Scoring a Custom Model
We use a generic term *custom model* to describe a model that was saved into a project as a file using a Python API and not WML API. 

While we recommend that models are saved using the WML API and deployed as *models* in Deployment Spaces, there may be cases when the models may need to be saved as files. Three typical reasons for using this approach are:
1. A model was trained outside of Watson Studio, but it needs to be deployed in Deployment Spaces
2. Framework used by the model is not supported by default in Watson Studio, that's why WML API can't be used. You can find the list of supported frameworks in documentation: https://www.ibm.com/docs/en/cloud-paks/cp-data/4.0?topic=functions-supported-deployment-frameworks
3. A customer has an established deployment process which works with exported model files. 

*In this notebook we demontrate how to load the model from the project (the model is available in Data Assets). You can also load the model file from any storage type - Storage Volume (shared file system), Git, Object Storage, etc.*

In [1]:
!pip install sklearn-pandas

### Step 1: Copy model to working directory

In [2]:
# If you ran the Predict_Customer_Churn notebook and did not shut down the environment before running this notebook, then the model will be in the directory
# If you shut down the environment, then the model will not be in the directory because the file system for the environment is not persisted
!ls

customer_churn_model_custom


In [3]:
from ibm_watson_studio_lib import access_project_or_space
wslib = access_project_or_space()

filename="customer_churn_model_custom"

# This command copies the file to the file system (working directory) of this environment
# We need to run this command becuase pickle API loads models from working directory
wslib.download_file(filename)

{'file_name': 'customer_churn_model_custom',
 'summary': ['loaded data', 'saved to file']}

In [5]:
# Verify that the model has been copied to the working directory
!ls -l

total 1728
-rw-rw----. 1 1000650000 wscommon 1766324 Dec  7 19:32 customer_churn_model_custom


### Step 2: Load and preview data that we will score

In [6]:
import itc_utils.flight_service as itcfs

readClient = itcfs.get_flight_client()

nb_data_request = {
    'data_name': """new_customers_with_id.csv""",
    'interaction_properties': {
    }
}

flightInfo = itcfs.get_flight_info(readClient, nb_data_request=nb_data_request)

scoring_input_data = itcfs.read_pandas_and_concat(readClient, flightInfo)
scoring_input_data.head(10)

ID Gender Status Children EstIncome CarOwner                 Age  \
0   1      F      S      0.0   77726.4        N               56.72   
1   2      M      S      2.0   63152.6        Y   63.17333299999999   
2   3      M      S      1.0   2056.06        Y               18.94   
3   4      F      M      1.0   20550.9        Y               22.12   
4   5      M      M      2.0   53010.8        N               18.84   
5   6      F      M      1.0   83287.0        Y                47.0   
6   7      F      M      2.0   76289.2        N  48.206666999999996   
7   8      F      M      2.0   56187.0        N                48.0   
8   9      F      D      1.0   85275.0        N                52.0   
9  10      F      S      0.0   17138.2        N           63.786667   

  AvgMonthlySpend CustomerSupportCalls Paymethod MembershipPlan  
0            9.51                  0.0      Auto            1.0  
1           40.07                  0.0        CC            2.0  
2           27.75                  0.0      Auto            4.0  
3           57.06                  0.0        CC            2.0  
4           46.42                  4.0        CC            1.0  
5           30.95                  0.0      Auto            1.0  
6           82.43                  0.0        CC            2.0  
7           13.74                  0.0        CH            2.0  
8          151.61                  0.0      Auto            3.0  
9           84.67                  0.0        CC            3.0

### Step 3: Load model and score data

In [10]:
import pickle

# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))

# Verify that the model was serialized into the right object
type(loaded_model)

sklearn.pipeline.Pipeline

In [26]:
# Get scoring results and probability of churn (prediciton of T=True)
scoring_results = loaded_model.predict(scoring_input_data)
scoring_prob_T = loaded_model.predict_proba(scoring_input_data)[:, 1]

In [28]:
# Scoring results are in an array format - the order is the same as the order of records in input, 
# and that's why we can append the input data with results based on order
print(scoring_results)

scoring_input_data['prediction']=scoring_results
scoring_input_data['probability_T'] = scoring_prob_T

[0 0 1 1 0 0 0 0 0 1 0 1 1 1 0 0 1 1 1 1 1 0 0 0 0 1 0 0 1 1 0 1 0 0 0 0 1
 1 1 0 1 0 1 0 0 1 0 1 0 1 1 0 0 1 0 1 1 0 1 0 0 1 1 0 1 1 0 0 1 0 0 1 0 1
 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 1 1 0 1 1 0 1 0 1 1 1 0 1 0 0 1 1 1 1 0 0 0
 0 0 1 0 1 1 0 1 1 1 1 1 1 1 0 0 1 1 1 0 1 1 0 0 0 0 1 0 1 0 0 1 1 1 1 1 1
 0 0 1 0 0 1 0 0 0 0 1 1 0 0 0 0 0 1 1 0 0 0 1 1 0 0 0 1 0 0 0 1 0 1 0 1 1
 1 1 0 1 0 0 1 1 0 0 0 0 0 0 0 1 1 0 0 1 0 1 1 1 0 1 0 0 1 0 1 1 0 0 1 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [29]:
scoring_output_data=scoring_input_data
# Display the data frame that we will write to a file
scoring_output_data

ID Gender Status Children EstIncome CarOwner                 Age  \
0      1      F      S      0.0   77726.4        N               56.72   
1      2      M      S      2.0   63152.6        Y   63.17333299999999   
2      3      M      S      1.0   2056.06        Y               18.94   
3      4      F      M      1.0   20550.9        Y               22.12   
4      5      M      M      2.0   53010.8        N               18.84   
..   ...    ...    ...      ...       ...      ...                 ...   
231  232      M      M      1.0   39229.1        N               26.26   
232  233      F      M      2.0   21419.9        N  59.833332999999996   
233  234      F      S      0.0   43382.3        N  16.073332999999998   
234  235      M      S      1.0   31952.0        N           34.266667   
235  236      F      M      2.0   98217.1        N               44.98   

    AvgMonthlySpend CustomerSupportCalls Paymethod MembershipPlan  prediction  \
0              9.51                  0.0      Auto            1.0           0   
1             40.07                  0.0        CC            2.0           0   
2             27.75                  0.0      Auto            4.0           1   
3             57.06                  0.0        CC            2.0           1   
4             46.42                  4.0        CC            1.0           0   
..              ...                  ...       ...            ...         ...   
231           13.53                  0.0        CC            4.0           0   
232          103.12                  0.0        CC            4.0           0   
233           99.94                  0.0        CH            3.0           0   
234            4.87                  2.0        CC            2.0           0   
235           13.24                  0.0        CC            4.0           0   

     probability_T  
0         0.092881  
1         0.122000  
2         0.715000  
3         0.736167  
4         0.060000  
..             ...  
231       0.309333  
232       0.102000  
233       0.291446  
234       0.275536  
235       0.000000  

[236 rows x 13 columns]

### Step 4: Save scoring results to Data Assets
In this example we save scoring results to Data Assets. You can also save scoring results to other data sources - Storage Volume (shared file system), Git, Object Storage, databases, Hadoop, etc. 

In [30]:
# First, we need to write the file to working directory
scoring_output_data.to_csv("ScoringOutput_Custom_Model.csv")

# upload the intermediate file to your Watson Studio project
wslib.upload_file(file_path="ScoringOutput_Custom_Model.csv", asset_name="ScoringOutput_Custom_Model.csv")

{'name': 'ScoringOutput_Custom_Model.csv',
 'asset_type': 'data_asset',
 'asset_id': 'f184f24e-334d-4d9c-aa8a-44110588306c',
 'attachment_id': '692f0608-4ca4-4744-940b-3c5aaff9df47',
 'filepath': 'ScoringOutput_Custom_Model.csv',
 'data_size': None,
 'mime': 'text/csv',
 'summary': ['created file in storage', 'created asset', 'created attachment'],
 'input_file_copied': True}

**Author:**  Elena Lowery and Catherine Cao <br/>
**Date:**  December 2021